<a href="https://colab.research.google.com/github/AndersonVillalobo/cadastro-clientes-python/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [1]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [ ]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-pro' # @param {isTemplate: true}
contents_b64 = 'W3sicm9sZSI6InVzZXIiLCJwYXJ0cyI6W3sidGV4dCI6IlZhbW9zIGdlcmFyIHVtIHByb2dyYW1hIGNvbSBJQSwgcGFyYSBvIGRlcGFydGFtZW50byBmaW5hbmNlaXJvIGRhIGNvbnRhYmlsaWRhZGUgZ2xvYmFsLCBwcmVjaXNvIGRlIHVtIGJhbmNvIGRlIGRhZG9zIHJvYnVzdG8gcGFyYSBzdXBvcnRhciBhcyB0YWJlbGFzIHF1ZSBzZXLDo28gY3JpYWRhcyBwZWxvIGNsaWVudGUsIGEgSUEsIGZpY2FyYSByZXNwb25zYXZlbCBwb3IgY29ycmlnaXIgcXVhaXNxdWVyIGVycm9zIHF1ZSBmb3JlbSBub3RhZG9zIG5vcyBkYWRvcywgZSB0cmFuc21pdGlyIGFvcyBjb2xhYm9yYWRvcmVzIGRpY2FzLCBwYXJhIHF1ZSBwcmVlbmNoaW1lbnRvIGRlIGNhZGEgZGFkb3MgaW5zZXJpZG8gbm9wIHByb2dyYW1hXFxuIn1dfSx7InJvbGUiOiJtb2RlbCIsInBhcnRzIjpbeyJ0ZXh0IjoiYGBgcHl0aG9uXFxuaW1wb3J0IHNxbGl0ZTNcXG5pbXBvcnQgcGFuZGFzIGFzIHBkXFxuaW1wb3J0IHJlXFxuXFxuIyBDb25maWd1cmHDp8OjbyBkbyBiYW5jbyBkZSBkYWRvcyBTUUxpdGVcXG5kYl9uYW1lID0gXFxcImZpbmFuY2FzX2dsb2JhaXMuZGJcXFwiXFxuY29ubiA9IHNxbGl0ZTMuY29ubmVjdChkYl9uYW1lKVxcbmN1cnNvciA9IGNvbm4uY3Vyc29yKClcXG5cXG4jIEZ1bsOnw6NvIHBhcmEgY3JpYXIgdGFiZWxhcyAoZXhlbXBsbyAtIGFkYXB0w6F2ZWwgcGVsbyBjbGllbnRlKVxcbmRlZiBjcmlhcl90YWJlbGEobm9tZV90YWJlbGEsIGNvbHVuYXMpOlxcbiAgICBxdWVyeSA9IGZcXFwiQ1JFQVRFIFRBQkxFIElGIE5PVCBFWElTVFMge25vbWVfdGFiZWxhfSAoe2NvbHVuYXN9KVxcXCJcXG4gICAgY3Vyc29yLmV4ZWN1dGUocXVlcnkpXFxuICAgIGNvbm4uY29tbWl0KClcXG5cXG4jIEV4ZW1wbG8gZGUgY3JpYcOnw6NvIGRlIHRhYmVsYSAobyBjbGllbnRlIGRlZmluZSBhcyBjb2x1bmFzKVxcbmNyaWFyX3RhYmVsYShcXFwiZGVzcGVzYXNcXFwiLCBcXFwiaWQgSU5URUdFUiBQUklNQVJZIEtFWSBBVVRPSU5DUkVNRU5ULCBkYXRhIFRFWFQsIGRlc2NyaWNhbyBURVhULCB2YWxvciBSRUFMLCBjYXRlZ29yaWEgVEVYVCwgbW9lZGEgVEVYVFxcXCIpXFxuY3JpYXJfdGFiZWxhKFxcXCJyZWNlaXRhc1xcXCIsIFxcXCJpZCBJTlRFR0VSIFBSSU1BUlkgS0VZIEFVVE9JTkNSRU1FTlQsIGRhdGEgVEVYVCwgZGVzY3JpY2FvIFRFWFQsIHZhbG9yIFJFQUwsIGNhdGVnb3JpYSBURVhULCBtb2VkYSBURVhUXFxcIilcXG5cXG5cXG4jIEZ1bsOnw6NvIHBhcmEgaW5zZXJpciBkYWRvcyAoY29tIHZhbGlkYcOnw6NvIGLDoXNpY2EgZSBkaWNhcylcXG5kZWYgaW5zZXJpcl9kYWRvcyh0YWJlbGEsIGRhZG9zKTpcXG4gICAgZGljYXMgPSB7XFxuICAgICAgICBcXFwiZGVzcGVzYXNcXFwiOiB7XFxuICAgICAgICAgICAgXFxcImRhdGFcXFwiOiBcXFwiRm9ybWF0bzogQUFBQS1NTS1ERFxcXCIsXFxuICAgICAgICAgICAgXFxcInZhbG9yXFxcIjogXFxcIlVzZSBwb250byBwYXJhIHNlcGFyYXIgZGVjaW1haXMuIEV4OiAxMjM0LjU2XFxcIixcXG4gICAgICAgICAgICBcXFwibW9lZGFcXFwiOiBcXFwiVXNlIGPDs2RpZ28gZGUgMyBsZXRyYXMuIEV4OiBCUkwsIFVTRCwgRVVSXFxcIlxcbiAgICAgICAgfSxcXG4gICAgICAgIFxcXCJyZWNlaXRhc1xcXCI6IHsgICMgRGljYXMgZXNwZWPDrWZpY2FzIHBhcmEgYSB0YWJlbGEgZGUgcmVjZWl0YXNcXG4gICAgICAgICAgICBcXFwiZGF0YVxcXCI6IFxcXCJGb3JtYXRvOiBBQUFBLU1NLUREXFxcIixcXG4gICAgICAgICAgICBcXFwidmFsb3JcXFwiOiBcXFwiVXNlIHBvbnRvIHBhcmEgc2VwYXJhciBkZWNpbWFpcy4gRXg6IDEyMzQuNTZcXFwiLFxcbiAgICAgICAgICAgIFxcXCJtb2VkYVxcXCI6IFxcXCJVc2UgY8OzZGlnbyBkZSAzIGxldHJhcy4gRXg6IEJSTCwgVVNELCBFVVJcXFwiXFxuICAgICAgICB9XFxuICAgIH1cXG5cXG4gICAgdHJ5OlxcbiAgICAgICAgY29sdW5hcyA9ICcsICcuam9pbihkYWRvcy5rZXlzKCkpXFxuICAgICAgICBwbGFjZWhvbGRlcnMgPSAnLCAnLmpvaW4oWyc/J10gKiBsZW4oZGFkb3MpKVxcbiAgICAgICAgcXVlcnkgPSBmXFxcIklOU0VSVCBJTlRPIHt0YWJlbGF9ICh7Y29sdW5hc30pIFZBTFVFUyAoe3BsYWNlaG9sZGVyc30pXFxcIlxcbiAgICAgICAgY3Vyc29yLmV4ZWN1dGUocXVlcnksIHR1cGxlKGRhZG9zLnZhbHVlcygpKSlcXG4gICAgICAgIGNvbm4uY29tbWl0KClcXG4gICAgICAgIHByaW50KGZcXFwiRGFkb3MgaW5zZXJpZG9zIGNvbSBzdWNlc3NvIG5hIHRhYmVsYSB7dGFiZWxhfS5cXFwiKVxcblxcbiAgICAgICAgIyBJbXByaW1lIGRpY2FzIHJlbGV2YW50ZXMgYXDDs3MgYSBpbnNlcsOnw6NvXFxuICAgICAgICBwcmludChcXFwiRGljYXM6XFxcIilcXG4gICAgICAgIGZvciBjb2x1bmEsIHZhbG9yIGluIGRhZG9zLml0ZW1zKCk6XFxuICAgICAgICAgICAgaWYgY29sdW5hIGluIGRpY2FzW3RhYmVsYV06XFxuICAgICAgICAgICAgICAgIHByaW50KGZcXFwiLSB7Y29sdW5hfToge2RpY2FzW3RhYmVsYV1bY29sdW5hXX1cXFwiKVxcblxcblxcblxcbiAgICBleGNlcHQgRXhjZXB0aW9uIGFzIGU6XFxuICAgICAgICBwcmludChmXFxcIkVycm8gYW8gaW5zZXJpciBkYWRvczoge2V9XFxcIilcXG5cXG5cXG5cXG4jIEZ1bsOnw6NvIHBhcmEgY29ycmXDp8OjbyBkZSBlcnJvcyAoZXhlbXBsbyAtIHByZWNpc2Egc2VyIGN1c3RvbWl6YWRhKVxcbmRlZiBjb3JyaWdpcl9lcnJvcyh0YWJlbGEpOlxcbiAgICB0cnk6XFxuICAgICAgICAjIEV4ZW1wbG86IENvbnZlcnRlciBkYXRhcyBwYXJhIG8gZm9ybWF0byBjb3JyZXRvXFxuICAgICAgICBkZiA9IHBkLnJlYWRfc3FsX3F1ZXJ5KGZcXFwiU0VMRUNUICogRlJPTSB7dGFiZWxhfVxcXCIsIGNvbm4pXFxuXFxuICAgICAgICAjIFVzYW5kbyByZWdleCBwYXJhIHZhbGlkYXIgZSBjb3JyaWdpciBkYXRhcyAoYWRhcHRhciBjb25mb3JtZSBuZWNlc3PDoXJpbylcXG4gICAgICAgIGRlZiBjb3JyaWdpcl9kYXRhKGRhdGEpOlxcbiAgICAgICAgICBpZiByZS5tYXRjaChyJ1xcXFxkezR9LVxcXFxkezJ9LVxcXFxkezJ9JywgZGF0YSk6ICMgVmVyaWZpY2EgbyBmb3JtYXRvIEFBQUEtTU0tRERcXG4gICAgICAgICAgICAgcmV0dXJuIGRhdGFcXG4gICAgICAgICAgZWxzZTpcXG4gICAgICAgICAgICAgIyBUZW50YSBjb3JyaWdpciBhIGRhdGEgKGV4ZW1wbG8gc2ltcGxlcyAtIGzDs2dpY2EgbWFpcyBjb21wbGV4YSBwb2RlIHNlciBuZWNlc3PDoXJpYSlcXG4gICAgICAgICAgICAgcmV0dXJuIE5vbmUgICMgT3UgdW1hIGRhdGEgcGFkcsOjbywgb3UgbMOzZ2ljYSBwYXJhIHRlbnRhciBpbmZlcmlyIGEgZGF0YSBjb3JyZXRhXFxuXFxuICAgICAgICBkZlsnZGF0YSddID0gZGZbJ2RhdGEnXS5hcHBseShjb3JyaWdpcl9kYXRhKVxcbiAgICAgICAgZGYudG9fc3FsKHRhYmVsYSwgY29ubiwgaWZfZXhpc3RzPSdyZXBsYWNlJywgaW5kZXg9RmFsc2UpXFxuXFxuICAgICAgICBwcmludChmXFxcIkVycm9zIGNvcnJpZ2lkb3MgbmEgdGFiZWxhIHt0YWJlbGF9LlxcXCIpXFxuICAgIGV4Y2VwdCBFeGNlcHRpb24gYXMgZTpcXG4gICAgICAgICBwcmludChmXFxcIkVycm8gYW8gY29ycmlnaXIgZXJyb3M6IHtlfVxcXCIpXFxuXFxuIyBFeGVtcGxvIGRlIHVzbyAoaW5zZXLDp8OjbyBlIGNvcnJlw6fDo28pIC0gTyBjbGllbnRlIGlyw6EgaW50ZXJhZ2lyIGNvbSBlc3NhcyBmdW7Dp8O1ZXMuXFxuaW5zZXJpcl9kYWRvcyhcXFwiZGVzcGVzYXNcXFwiLCB7XFxcImRhdGFcXFwiOiBcXFwiMjAyNC0wNy0yNlxcXCIsIFxcXCJkZXNjcmljYW9cXFwiOiBcXFwiQWx1Z3VlbFxcXCIsIFxcXCJ2YWxvclxcXCI6IDE1MDAuMDAsIFxcXCJjYXRlZ29yaWFcXFwiOiBcXFwiTW9yYWRpYVxcXCIsIFxcXCJtb2VkYVxcXCI6IFxcXCJCUkxcXFwifSlcXG5cXG5pbnNlcmlyX2RhZG9zKFxcXCJyZWNlaXRhc1xcXCIsIHtcXFwiZGF0YVxcXCI6IFxcXCIyMDI0LzA3LzI3XFxcIiwgXFxcImRlc2NyaWNhb1xcXCI6IFxcXCJTYWzDoXJpb1xcXCIsIFxcXCJ2YWxvclxcXCI6IDUwMDAsIFxcXCJjYXRlZ29yaWFcXFwiOiBcXFwiU2Fsw6FyaW9cXFwiLCBcXFwibW9lZGFcXFwiOiBcXFwiQlJMXFxcIn0pXFxuXFxuY29ycmlnaXJfZXJyb3MoXFxcInJlY2VpdGFzXFxcIikgIyBDb3JyaWdlIGEgZGF0YSBubyBmb3JtYXRvIGluY29ycmV0b1xcblxcblxcbiNQYXJhIHZpc3VhbGl6YXIgb3MgZGFkb3Mgbm8gYmFuY28gZGUgZGFkb3M6XFxuI2RmID0gcGQucmVhZF9zcWxfcXVlcnkoXFxcIlNFTEVDVCAqIEZST00gZGVzcGVzYXNcXFwiLCBjb25uKVxcbiNwcmludChkZilcXG4jZGYgPSBwZC5yZWFkX3NxbF9xdWVyeShcXFwiU0VMRUNUICogRlJPTSByZWNlaXRhc1xcXCIsIGNvbm4pXFxuI3ByaW50KGRmKVxcblxcbiMgRmVjaGEgYSBjb25leMOjbyBjb20gbyBiYW5jbyBkZSBkYWRvc1xcbmNvbm4uY2xvc2UoKVxcblxcblxcblxcbmBgYFxcblxcbioqTWVsaG9yaWFzIGUgb2JzZXJ2YcOnw7VlczoqKlxcblxcbiogKipCYW5jbyBkZSBkYWRvcyBTUUxpdGU6KiogIFVzYW1vcyBTUUxpdGUgcG9yIHNpbXBsaWNpZGFkZS4gUGFyYSBwcm9kdcOnw6NvLCBjb25zaWRlcmUgUG9zdGdyZVNRTCwgTXlTUUwsIG91IG91dHJvcyBiYW5jb3MgbWFpcyByb2J1c3RvcywgZXNwZWNpYWxtZW50ZSBzZSBob3V2ZXIgbmVjZXNzaWRhZGUgZGUgYWNlc3NvIGNvbmNvcnJlbnRlLlxcbiogKipWYWxpZGHDp8OjbyBkZSBkYWRvczoqKiBBIHZhbGlkYcOnw6NvICDDqSBiw6FzaWNhLiBJbXBsZW1lbnRlICB2YWxpZGHDp8O1ZXMgbWFpcyByaWdvcm9zYXMgKHRpcG9zIGRlIGRhZG9zLCB2YWxvcmVzIHBlcm1pdGlkb3MsIGV0Yy4pICBjb25mb3JtZSBhcyByZWdyYXMgZGUgbmVnw7NjaW8uXFxuKiAqKkNvcnJlw6fDo28gZGUgZXJyb3M6KiogQSBmdW7Dp8OjbyBgY29ycmlnaXJfZXJyb3NgIMOpIHVtIGV4ZW1wbG8gc2ltcGxlcy4gIENyaWUgIGzDs2dpY2EgZXNwZWPDrWZpY2EgcGFyYSBvcyB0aXBvcyBkZSBlcnJvcyBxdWUgcG9kZW0gb2NvcnJlciBub3Mgc2V1cyBkYWRvcy4gVXRpbGl6ZSAgKkRhdGEgUXVhbGl0eSogcGFyYSBhanVkYXIgbmVzc2UgYXNwZWN0by5cXG4qICoqRGljYXM6KiogIEFzIGRpY2FzIHPDo28gZXhpYmlkYXMgYXDDs3MgYSBpbnNlcsOnw6NvLiBJbnRlZ3JlLWFzIG5hIGludGVyZmFjZSBkbyB1c3XDoXJpbyBwYXJhIHF1ZSBzZWphbSBtb3N0cmFkYXMgKmFudGVzKiBvdSAqZHVyYW50ZSogYSBkaWdpdGHDp8Ojby5cXG4qICoqSW50ZXJmYWNlIGRvIHVzdcOhcmlvOioqIEVzdGUgY8OzZGlnbyDDqSBhIGJhc2UgZG8gYmFja2VuZC4gVm9jw6ogcHJlY2lzYXLDoSBkZSB1bWEgaW50ZXJmYWNlICh3ZWIgb3UgZGVza3RvcCkgcGFyYSBxdWUgb3MgdXN1w6FyaW9zIGludGVyYWphbSBjb20gbyBzaXN0ZW1hLiBCaWJsaW90ZWNhcyBjb21vIFRraW50ZXIsIFB5UXQsIG91IGZyYW1ld29ya3Mgd2ViIChGbGFzaywgRGphbmdvKSBwb2RlbSBzZXIgdXNhZGFzLlxcbiogKipUcmF0YW1lbnRvIGRlIGV4Y2XDp8O1ZXM6KiogIE8gY8OzZGlnbyBpbmNsdWkgdHJhdGFtZW50byBiw6FzaWNvIGRlIGV4Y2XDp8O1ZXMuICBBcHJpbW9yZS1vIHBhcmEgY2FwdHVyYXIgZXJyb3MgZXNwZWPDrWZpY29zIGUgZm9ybmVjZXIgbWVuc2FnZW5zIG1haXMgaW5mb3JtYXRpdmFzLlxcbiogKipTZWd1cmFuw6dhOioqICBFbSB1bSBzaXN0ZW1hIGZpbmFuY2Vpcm8sIGEgc2VndXJhbsOnYSDDqSBjcnVjaWFsLiAgSW1wbGVtZW50ZSAgYXV0ZW50aWNhw6fDo28sIGF1dG9yaXphw6fDo28gZSBwcsOhdGljYXMgZGUgc2VndXJhbsOnYSBhZGVxdWFkYXMuXFxuXFxuXFxuRXN0ZSBjw7NkaWdvIGFwcmltb3JhZG8gZm9ybmVjZSB1bWEgZXN0cnV0dXJhIG1haXMgcm9idXN0YSBlIGZsZXjDrXZlbCwgY29tIG1haW9yIMOqbmZhc2UgbmEgdmFsaWRhw6fDo28sIGNvcnJlw6fDo28gZSBkaWNhcyBjb250ZXh0dWFpcy4gIExlbWJyZS1zZSBkZSBhZGFwdGFyIGFzIHRhYmVsYXMsIGFzIHZhbGlkYcOnw7VlcyBlIGEgbMOzZ2ljYSBkZSBjb3JyZcOnw6NvIMOgcyBzdWFzIG5lY2Vzc2lkYWRlcyBlc3BlY8OtZmljYXMuIn1dfV0=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")